# Trial End ~ all frequencies: BJH046

In [15]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal, stats
import re
import os
import mne
import IPython
import seaborn as sns
import scipy
import joblib
import h5io
import dask.array as da 

import statsmodels
from statsmodels import stats
from statsmodels.stats import multitest

## import custom functions
import sys
sys.path.append('/home/brooke/pacman/preprocessing/scripts')
import preproc_functions as pf
import roi as subs


In [ ]:
## Prep paths ##

subject = 'BJH046'
raw_data_dir = f"/home/brooke/pacman/raw_data/{subject}"
preproc_data_dir = f"/home/brooke/pacman/preprocessing/{subject}/ieeg"

In [ ]:
## Load Neural Data

# load
trial_end_epochs = mne.read_epochs(f"{preproc_data_dir}/{subject}_bp_clean_end-locked_ieeg.fif")

# get good epochs (for behavioral data only)
good_epochs = [i for i,x in enumerate(trial_end_epochs.get_annotations_per_epoch()) if not x]
bad_epochs = [i for i,x in enumerate(trial_end_epochs.get_annotations_per_epoch()) if  x]

# load behavioral data
trial_end_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_died_events.csv")

# set info as metadata
trial_end_epochs.metadata = trial_end_data

# onlt good epochs
trial_end_epochs = trial_end_epochs[good_epochs]

In [ ]:
## Dictionary of electrode locations ##

# Pull mapping ROI to elecs
ROIs = subs.ROIs[subject]

## prep lists

# limbic ROI
hc_list = []
hc_indices = []
hc_names = []
ofc_list = []
ofc_indices = []
ofc_names = []
amyg_list = []
amyg_names = [] 
amyg_indices = []
cing_list = []
cing_names = [] 
cing_indices = []
sgACC_list = []
sgACC_names = [] 
sgACC_indices = []
dACC_list = []
dACC_names = [] 
dACC_indices = []
sfg_list = []
sfg_names = [] 
sfg_indices = []
mfg_list = []
mfg_names = [] 
mfg_indices = []

# control ROI
insula_list = []
insula_names = []  
insula_indices = []
dlpfc_list = []
dlpfc_names = []  
dlpfc_indices = []
ec_list = []
ec_names = []  
ec_indices = []

# exclude bad ROI from list
pairs_long_name = [ch.split('-') for ch in trial_end_epochs.pick_types(seeg=True).info['ch_names']]
bidx = len(trial_end_epochs.info['bads']) +1
pairs_name = pairs_long_name[bidx:len(pairs_long_name)]

# sort ROI into lists
for ix in range(0, len(pairs_name)):
    if pairs_name[ix][0] in ROIs['hc'] or pairs_name[ix][1] in ROIs['hc']:
        hc_list.append(trial_end_epochs.info['ch_names'][ix + bidx])
        hc_names.append(pairs_name[ix])
        hc_indices.append(ix)
    if pairs_name[ix][0] in ROIs['ofc'] or pairs_name[ix][1] in ROIs['ofc']:
        ofc_list.append(trial_end_epochs.info['ch_names'][ix + bidx])
        ofc_names.append(pairs_name[ix])
        ofc_indices.append(ix)
    if pairs_name[ix][0] in ROIs['amyg'] or pairs_name[ix][1] in ROIs['amyg']:
        amyg_list.append(trial_end_epochs.info['ch_names'][ix + bidx])       
        amyg_names.append(pairs_name[ix])
        amyg_indices.append(ix)
    if pairs_name[ix][0] in ROIs['cing'] or pairs_name[ix][1] in ROIs['cing']:
        cing_list.append(trial_end_epochs.info['ch_names'][ix + bidx])       
        cing_names.append(pairs_name[ix])
        cing_indices.append(ix)
    if pairs_name[ix][0] in ROIs['sgACC'] or pairs_name[ix][1] in ROIs['sgACC']:
        sgACC_list.append(trial_end_epochs.info['ch_names'][ix + bidx])       
        sgACC_names.append(pairs_name[ix])
        sgACC_indices.append(ix)
    if pairs_name[ix][0] in ROIs['dACC'] or pairs_name[ix][1] in ROIs['dACC']:
        dACC_list.append(trial_end_epochs.info['ch_names'][ix + bidx])       
        dACC_names.append(pairs_name[ix])
        dACC_indices.append(ix)        
        
    # control roi
    if pairs_name[ix][0] in ROIs['insula'] or pairs_name[ix][1] in ROIs['insula']:
        insula_list.append(trial_end_epochs.info['ch_names'][ix + bidx])       
        insula_names.append(pairs_name[ix])
        insula_indices.append(ix)
    if pairs_name[ix][0] in ROIs['dlpfc'] or pairs_name[ix][1] in ROIs['dlpfc']:
        dlpfc_list.append(trial_end_epochs.info['ch_names'][ix + bidx])       
        dlpfc_names.append(pairs_name[ix])
        dlpfc_indices.append(ix)
    if pairs_name[ix][0] in ROIs['sfg'] or pairs_name[ix][1] in ROIs['sfg']:
        sfg_list.append(trial_end_epochs.info['ch_names'][ix + bidx])       
        sfg_names.append(pairs_name[ix])
        sfg_indices.append(ix)   
    if pairs_name[ix][0] in ROIs['mfg'] or pairs_name[ix][1] in ROIs['mfg']:
        mfg_list.append(trial_end_epochs.info['ch_names'][ix + bidx])       
        mfg_names.append(pairs_name[ix])
        mfg_indices.append(ix)     
    if pairs_name[ix][0] in ROIs['ec'] or pairs_name[ix][1] in ROIs['ec']:
        ec_list.append(trial_end_epochs.info['ch_names'][ix + bidx])       
        ec_names.append(pairs_name[ix])
        ec_indices.append(ix)        
        

In [ ]:
# Print and check frequencies
print(pf.freqs)
print(pf.n_cycles)
print(pf.time_bin)
print(pf.band_width)


# Main Regions of Interest

## Hippocampus

In [ ]:
hc_list

In [ ]:
# only ROI of interest
trial_end_roi = trial_end_epochs.copy().pick_channels(hc_list)


In [ ]:
# Resample to 1000 
if trial_end_roi.info['sfreq'] > 1000:
    trial_end_roi= trial_end_roi.resample(1000)

In [ ]:
# compute TRF

roi_tfr = []
roi_tfr = pf.compute_TFR(trial_end_roi,pf.freqs,pf.n_cycles)


In [ ]:
# Crop to time of interest

roi_tfr.crop(tmin = -2.5, tmax = 2.5)

In [ ]:
# save TFRs

roi_tfr.save(f"/home/brooke/pacman/preprocessing/{subject}/ieeg/trial_end/hc-tfr.h5", overwrite = True)

In [ ]:
# Log and zscore

roi_tfr = pf.log_and_zscore_TFR(roi_tfr, baseline = (-2.5,2.5), logflag=True)

In [ ]:
# Plot TFR - Escape
pf.plot_average_tfr(roi_tfr['chase_trial == 0 & died == 0'], f"Average Hippocampal TFR at Trial End: Escape, n={len(hc_list)}", subject, 'hc_escape')


In [ ]:
# Plot TFR - Chase
pf.plot_average_tfr(roi_tfr['chase_trial == 1 & died == 0'], f"Average Hippocampal TFR at Trial End: Chase, n={len(hc_list)}", subject, 'hc_chase')


In [ ]:
# Plot TFR - Died
pf.plot_average_tfr(roi_tfr['died == 1'], f"Average Hippocampal TFR at Trial End: Died, n={len(hc_list)}", subject, 'hc_died')


## Individual Channels

In [ ]:
# Plot Channel TFR
for chix, ch in enumerate(roi_tfr.ch_names):

    pf.plot_channel_tfr(roi_tfr['chase_trial == 0 & died == 0'], chix, ch, 'Trial End - Conflict')
    pf.plot_channel_tfr(roi_tfr['chase_trial == 1 & died == 0'], chix, ch, 'Trial End - Chase')
    pf.plot_channel_tfr(roi_tfr['died == 1'], chix, ch, 'Trial End - Died')
    


# OFC

In [ ]:
ofc_list

In [ ]:
# only roi of interest
trial_end_roi = trial_end_epochs.copy().pick_channels(ofc_list)


In [ ]:
# Resample to 1000 
if trial_end_roi.info['sfreq'] > 1000:
    trial_end_roi= trial_end_roi.resample(1000)

In [ ]:
# compute TRF

roi_tfr = []
roi_tfr = pf.compute_TFR(trial_end_roi,pf.freqs,pf.n_cycles)

In [ ]:
roi_tfr.crop(tmin = -2.5, tmax = 2.5)


In [ ]:
# save TFRs

roi_tfr.save(f"/home/brooke/pacman/preprocessing/{subject}/ieeg/trial_end/ofc-tfr.h5", overwrite = True)

In [ ]:
# Log and zscore

roi_tfr = pf.log_and_zscore_TFR(roi_tfr, baseline = (-2.5,2.5), logflag=True)

In [ ]:
# Plot TFR - Escape
pf.plot_average_tfr(roi_tfr['chase_trial == 0 & died == 0'], f"Average OFC TFR at Trial End: Escape, n={len(ofc_list)}", subject, 'ofc_escape')


In [ ]:
# Plot TFR - Chase
pf.plot_average_tfr(roi_tfr['chase_trial == 1 & died == 0'], f"Average OFC TFR at Trial End: Chase, n={len(ofc_list)}", subject, 'ofc_chase')


In [ ]:
# Plot TFR - Died
pf.plot_average_tfr(roi_tfr['died == 1'], f"Average OFC TFR at Trial End: Died, n={len(ofc_list)}", subject, 'ofc_died')


### Individual Channels

In [ ]:
# Plot Channel TFR
for chix, ch in enumerate(roi_tfr.ch_names):

    pf.plot_channel_tfr(roi_tfr['chase_trial == 0 & died == 0'], chix, ch, 'Trial End - Conflict')
    pf.plot_channel_tfr(roi_tfr['chase_trial == 1 & died == 0'], chix, ch, 'Trial End - Chase')
    pf.plot_channel_tfr(roi_tfr['died == 1'], chix, ch, 'Trial End - Died')

# Amygdala

In [ ]:
amyg_list

In [ ]:
# only roi 
trial_end_roi = trial_end_epochs.copy().pick_channels(amyg_list)


In [ ]:
# Resample to 1000 
if trial_end_roi.info['sfreq'] > 1000:
    trial_end_roi= trial_end_roi.resample(1000)

In [ ]:
# compute TRF

roi_tfr = []
roi_tfr = pf.compute_TFR(trial_end_roi,pf.freqs,pf.n_cycles)


In [ ]:
roi_tfr.crop(tmin = -2.5, tmax = 2.5)


In [ ]:
# save TFRs

roi_tfr.save(f"/home/brooke/pacman/preprocessing/{subject}/ieeg/trial_end/amyg-tfr.h5", overwrite = True)

In [ ]:
# Log and zscore

roi_tfr = pf.log_and_zscore_TFR(roi_tfr, baseline = (-2.5,2.5), logflag=True)

In [ ]:
# Plot TFR - Escape
pf.plot_average_tfr(roi_tfr['chase_trial == 0 & died == 0'], f"Average Amygdala TFR at Trial End: Escape, n={len(amyg_list)}", subject, 'amyg_escape')


In [ ]:
# Plot TFR - Chase
pf.plot_average_tfr(roi_tfr['chase_trial == 1 & died == 0'], f"Average Amygdala TFR at Trial End: Chase, n={len(amyg_list)}", subject, 'amyg_chase')


In [ ]:
# Plot TFR - Died
pf.plot_average_tfr(roi_tfr['died == 1'], f"Average Amygdala TFR at Trial End: Died, n={len(amyg_list)}", subject, 'amyg_died')


### Individual Channels

In [ ]:
# Plot Channel TFR
for chix, ch in enumerate(roi_tfr.ch_names):

    pf.plot_channel_tfr(roi_tfr['chase_trial == 0 & died == 0'], chix, ch, 'Trial End - Conflict')
    pf.plot_channel_tfr(roi_tfr['chase_trial == 1 & died == 0'], chix, ch, 'Trial End - Chase')
    pf.plot_channel_tfr(roi_tfr['died == 1'], chix, ch, 'Trial End - Died')

## Cingulate

In [ ]:
cing_list

In [ ]:
# only roi 
trial_end_roi = trial_end_epochs.copy().pick_channels(cing_list)


In [ ]:
# Resample to 1000 
if trial_end_roi.info['sfreq'] > 1000:
    trial_end_roi= trial_end_roi.resample(1000)

In [ ]:
# compute TRF

roi_tfr = []
roi_tfr = pf.compute_TFR(trial_end_roi,pf.freqs,pf.n_cycles)


In [ ]:
roi_tfr.crop(tmin = -2.5, tmax = 2.5)


In [ ]:
# save TFRs

roi_tfr.save(f"/home/brooke/pacman/preprocessing/{subject}/ieeg/trial_end/cing-tfr.h5", overwrite = True)

In [ ]:
# Log and zscore

roi_tfr = pf.log_and_zscore_TFR(roi_tfr, baseline = (-2.5,2.5), logflag=True)

In [ ]:
# Plot TFR - Escape
pf.plot_average_tfr(roi_tfr['chase_trial == 0 & died == 0'], f"Average Ant. Cingulate TFR at Trial End: Escape, n={len(cing_list)}", subject, 'cing_escape')


In [ ]:
# Plot TFR - Chase
pf.plot_average_tfr(roi_tfr['chase_trial == 1 & died == 0'], f"Average Ant. Cingulate TFR at Trial End: Chase, n={len(cing_list)}", subject, 'cing_chase')


In [ ]:
# Plot TFR - Died
pf.plot_average_tfr(roi_tfr['died == 1'], f"Average Ant. Cingulate TFR at Trial End: Died, n={len(cing_list)}", subject, 'cing_died')


### Individual Channels

In [ ]:
# Plot Channel TFR
for chix, ch in enumerate(roi_tfr.ch_names):

    pf.plot_channel_tfr(roi_tfr['chase_trial == 0 & died == 0'], chix, ch, 'Trial End - Conflict')
    pf.plot_channel_tfr(roi_tfr['chase_trial == 1 & died == 0'], chix, ch, 'Trial End - Chase')
    pf.plot_channel_tfr(roi_tfr['died == 1'], chix, ch, 'Trial End - Died')

# Control Regions

## Insula

In [ ]:
insula_list

In [ ]:
# Only ROI
trial_end_roi = trial_end_epochs.copy().pick_channels(insula_list)


In [ ]:
# Resample to 1000 
if trial_end_roi.info['sfreq'] > 1000:
    trial_end_roi= trial_end_roi.resample(1000)

In [ ]:
# compute TRF

roi_tfr = []
roi_tfr = pf.compute_TFR(trial_end_roi,pf.freqs,pf.n_cycles)


In [ ]:
roi_tfr.crop(tmin = -2.5, tmax = 2.5)


In [ ]:
# save TFRs

roi_tfr.save(f"/home/brooke/pacman/preprocessing/{subject}/ieeg/trial_end/insula-tfr.h5", overwrite = True)

In [ ]:
# Log and zscore

roi_tfr = pf.log_and_zscore_TFR(roi_tfr, baseline = (-2.5,2.5), logflag=True)

In [ ]:
# Plot TFR - Escape
pf.plot_average_tfr(roi_tfr['chase_trial == 0 & died == 0'], f"Average Insula TFR at Trial End: Escape, n={len(insula_list)}", subject, 'insula_escape')


In [ ]:
# Plot TFR - Chase
pf.plot_average_tfr(roi_tfr['chase_trial == 1 & died == 0'], f"Average Insula TFR at Trial End: Chase, n={len(insula_list)}", subject, 'insula_chase')


In [ ]:
# Plot TFR - Died
pf.plot_average_tfr(roi_tfr['died == 1'], f"Average Insula TFR at Trial End: Died, n={len(insula_list)}", subject, 'insula_died')


### Individual Channels

In [ ]:
# Plot Channel TFR
for chix, ch in enumerate(roi_tfr.ch_names):

    pf.plot_channel_tfr(roi_tfr['chase_trial == 0 & died == 0'], chix, ch, 'Trial End - Conflict')
    pf.plot_channel_tfr(roi_tfr['chase_trial == 1 & died == 0'], chix, ch, 'Trial End - Chase')
    pf.plot_channel_tfr(roi_tfr['died == 1'], chix, ch, 'Trial End - Died')

## dlPFC

In [ ]:
dlpfc_list

In [ ]:
# Only ROI
trial_end_roi = trial_end_epochs.copy().pick_channels(dlpfc_list)


In [ ]:
# Resample to 1000 
if trial_end_roi.info['sfreq'] > 1000:
    trial_end_roi= trial_end_roi.resample(1000)

In [ ]:
# compute TRF

roi_tfr = []
roi_tfr = pf.compute_TFR(trial_end_roi,pf.freqs,pf.n_cycles, workers = 2)

In [ ]:
roi_tfr.crop(tmin = -2.5, tmax = 2.5)


In [ ]:
# save TFRs

roi_tfr.save(f"/home/brooke/pacman/preprocessing/{subject}/ieeg/trial_end/dlpfc-tfr.h5", overwrite = True)

In [ ]:
# Log and zscore

roi_tfr = pf.log_and_zscore_TFR(roi_tfr, baseline = (-2.5,2.5), logflag=True)

In [ ]:
# Plot TFR - Escape
pf.plot_average_tfr(roi_tfr['chase_trial == 0 & died == 0'], f"Average dlPFC TFR at Trial End: Escape, n={len(dlpfc_list)}", subject, 'dlpfc_escape')


In [ ]:
# Plot TFR - Chase
pf.plot_average_tfr(roi_tfr['chase_trial == 1 & died == 0'], f"Average dlPFC TFR at Trial End: Chase, n={len(dlpfc_list)}", subject, 'dlpfc_chase')


In [ ]:
# Plot TFR - Died
pf.plot_average_tfr(roi_tfr['died == 1'], f"Average dlPFC TFR at Trial End: Died, n={len(dlpfc_list)}", subject, 'dlpfc_died')


### Individual Channels

In [ ]:
# Plot Channel TFR
for chix, ch in enumerate(roi_tfr.ch_names):

    pf.plot_channel_tfr(roi_tfr['chase_trial == 0 & died == 0'], chix, ch, 'Trial End - Conflict')
    pf.plot_channel_tfr(roi_tfr['chase_trial == 1 & died == 0'], chix, ch, 'Trial End - Chase')
    pf.plot_channel_tfr(roi_tfr['died == 1'], chix, ch, 'Trial End - Died')

## MFG

In [ ]:
mfg_list

In [ ]:
# Plot TFR - Escape
pf.plot_average_tfr(roi_tfr['chase_trial == 0 & died == 0'].pick_channels(mfg_list), f"Average MFG TFR at Trial End: Escape, n={len(mfg_list)}", subject, 'mfg_escape')


In [ ]:
# Plot TFR - Chase
pf.plot_average_tfr(roi_tfr['chase_trial == 1 & died == 0'].pick_channels(mfg_list), f"Average MFG TFR at Trial End: Chase, n={len(mfg_list)}", subject, 'mfg_chase')


In [ ]:
# Plot TFR - Died
pf.plot_average_tfr(roi_tfr['died == 1'], f"Average MFG TFR at Trial End: Died, n={len(mfg_list)}", subject, 'mfg_died')


### Individual Channels

In [ ]:
plt.rcParams['figure.figsize'] = [15, 11]

for ch in mfg_list:
    chix = dlpfc_list.index(ch)
    pf.plot_channel_tfr(roi_tfr['chase_trial == 0 & died == 0'], chix, ch, 'Trial End - Conflict')
    pf.plot_channel_tfr(roi_tfr['chase_trial == 1 & died == 0'], chix, ch, 'Trial End - Chase')
    pf.plot_channel_tfr(roi_tfr['died == 1'], chix, ch, 'Trial End - Died')
    

## SFG

In [ ]:
sfg_list

In [ ]:
# Plot TFR - Escape
pf.plot_average_tfr(roi_tfr['chase_trial == 0 & died == 0'].pick_channels(sfg_list), f"Average SFG TFR at Trial End: Escape, n={len(sfg_list)}", subject, 'sfg_escape')


In [ ]:
# Plot TFR - Chase
pf.plot_average_tfr(roi_tfr['chase_trial == 1 & died == 0'].pick_channels(sfg_list), f"Average SFG TFR at Trial End: Chase, n={len(sfg_list)}", subject, 'sfg_chase')


In [ ]:
# Plot TFR - Died
pf.plot_average_tfr(roi_tfr['died == 1'], f"Average SFG TFR at Trial End: Died, n={len(sfg_list)}", subject, 'sfg_died')


### Individual Channels

In [ ]:
plt.rcParams['figure.figsize'] = [15, 11]

for ch in sfg_list:
    chix = dlpfc_list.index(ch)
    pf.plot_channel_tfr(roi_tfr['chase_trial == 0 & died == 0'], chix, ch, 'Trial End - Conflict')
    pf.plot_channel_tfr(roi_tfr['chase_trial == 1 & died == 0'], chix, ch, 'Trial End - Chase')
    pf.plot_channel_tfr(roi_tfr['died == 1'], chix, ch, 'Trial End - Died')